In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from html.parser import HTMLParser
import re
import os
import nltk
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import pickle
from sklearn.svm import SVR

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def handle_starttag(self, tag, attrs):
        if(tag=='a'):
            for (att, val) in attrs:
                if (att=='href'):
                    self.fed.append(' hreflink ')
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()


vocab = []
word_list = list(set(words.words()))
labels = []

def read_unlabeled_email(filename):
    try:
        with open(filename, 'r') as File:  
            return re.sub(r'[^a-zA-Z]', ' ', strip_tags(re.sub(r'http\S+', ' hreflink ', File.read()))).strip().lower()
    except UnicodeDecodeError:
        print('couldnt read {!s}',filename)
        return None
def read_email(filename,label):
    print(filename)
    try:
        with open(filename, 'r') as File:  
            
            email = re.sub(r'[^a-zA-Z]', ' ', strip_tags(re.sub(r'http\S+', ' hreflink ', File.read()))).strip().lower()
            for word in word_tokenize(email):
                if (word in word_list) and (len(word) >= 2):
                    vocab.append(word)
            labels.append(label)
            return email
    except UnicodeDecodeError:
        print('couldnt read {!s}',filename)
        return None


corpus = []
spam_train_path = './training/spam/'
for filename in os.listdir(spam_train_path):
    em = read_email((spam_train_path+filename),1)
    if (em is None):
        continue
    corpus.append(em)
    
    
ham_train_path = './training/ham/'
for filename in os.listdir(ham_train_path):
    em = read_email((ham_train_path+filename),0)
    if (em is None):
        continue
    corpus.append(em)


vocab = list(set(vocab))
print(len(vocab))

#saving vars
f = open('./vars/corpus.pckl', 'wb')
pickle.dump(corpus, f)
f.close()

f = open('./vars/vocab.pckl', 'wb')
pickle.dump(vocab, f)
f.close()

f = open('./vars/labels.pckl', 'wb')
pickle.dump(labels, f)
f.close()


    




./training/spam/00001.317e78fa8ee2f54cd4890fdc09ba8176
./training/spam/00002.9438920e9a55591b18e60d1ed37d992b
./training/spam/00003.590eff932f8704d8b0fcbe69d023b54d
./training/spam/00004.bdcc075fa4beb5157b5dd6cd41d8887b
./training/spam/00005.ed0aba4d386c5e62bc737cf3f0ed9589
./training/spam/00006.3ca1f399ccda5d897fecb8c57669a283
./training/spam/00007.acefeee792b5298f8fee175f9f65c453
./training/spam/00008.ccf927a6aec028f5472ca7b9db9eee20
./training/spam/00009.1e1a8cb4b57532ab38aa23287523659d
./training/spam/00010.2558d935f6439cb40d3acb8b8569aa9b
./training/spam/00011.bd8c904d9f7b161a813d222230214d50
./training/spam/00012.cb9c9f2a25196f5b16512338625a85b4
./training/spam/00013.372ec9dc663418ca71f7d880a76f117a
./training/spam/00014.13574737e55e51fe6737a475b88b5052
./training/spam/00015.206d5a5d1d34272ae32fc286788fdf55
./training/spam/00016.4fb07c8dff1a5a2b4889dc5024c55023
./training/spam/00017.6430f3b8dedf51ba3c3fcb9304e722e7
./training/spam/00018.336cb9e7b0358594cf002e7bf669eaf5
./training

./training/spam/00150.957bb781217f762dc9999e7a90130c92
./training/spam/00151.6abbf42bc1bfb6c36b749372da0cffae
./training/spam/00152.a9f16de7f087215259a15322961bf9c0
./training/spam/00153.d20d157c684520f1c3aa8f270f753785
./training/spam/00154.fb13b55bdbb01e81ac9b8ee6f13948d5
./training/spam/00155.b043f6801a72ada945205709c44c8ac4
./training/spam/00156.ccb4ccdec1949b9510ab71d05122de3f
./training/spam/00157.eab255ce291e88fbee46f3f0c564ddf3
./training/spam/00158.58aea46256aaaa23787ec2acdcd31073
./training/spam/00159.6b641c70d79fd5a69b84a94b4e88150a
./training/spam/00160.66e11a17d619ce33d5a455a87015ee25
./training/spam/00161.37836bbc77bdab253f914d7a5233badb
./training/spam/00162.b5ae5521352c9bba7bf635c1766d4a75
./training/spam/00163.2ceade6f8b0c1c342f5f95d57ac551f5
./training/spam/00164.272880ebd1f1f93cf0cd9800842a24bd
./training/spam/00165.3a37220d69b5b8332ee4270f0121dfe9
./training/spam/00166.806d5398d7a37c080641a2d62e2d2b94
./training/spam/00167.c37f166df54b89ce15059415cfbe12c3
./training

./training/spam/00299.ec4bd0c57a7bf6a5616beb2897aaed7b
./training/spam/00300.6ca4fee75f6afbd00581ceec6cf14fac
./training/spam/00301.3b6fa92db458408d9468360fc034d280
./training/spam/00302.7f695c69d4cbe06e91cddd2ca7cddf33
./training/spam/00303.7d749e4a46ceb169ea1af5b9e5ab39a9
./training/spam/00304.fedb2050801439ca46d1d82d10e8e9e2
./training/spam/00305.1f2a712e25638b0a4868155b37022cf1
./training/spam/00306.9e12acdecaf3825733a1aadc2455b166
./training/spam/00307.79b64580c5c605583aec7b7a4f8679c0
./training/spam/00308.fc90f8aab51648329b9e705c9021b204
./training/spam/00309.514ba73d47cc5668a2afdef0a25b400c
./training/spam/00310.dd799961317914374088b04cc0fb1b85
./training/spam/00311.176626eb0ec0de8f451a079083104975
./training/spam/00312.5034fee7d265abd7a4194e9300de24bf
./training/spam/00313.4363902393c0037670bc9483d19dc551
./training/spam/00314.ce1926a9815415807e51b80930bffdb8
./training/spam/00315.81905f8867f52179286a6a5bdd1483fa
./training/spam/00316.6127940652124130611907ee0c20ab5e
./training

./training/spam/00448.65d06c45ea553e3fddf51645ae6b07f0
./training/spam/00449.c498bdd182edba9e9ba725c5a5a1f06b
./training/spam/00450.acfa2d7f64e43ef04600e30fdecff8ec
./training/spam/00451.bbff6de62f0340d64a044870dbedafba
./training/spam/00452.f13574a4582c94daf2bd6668c1683eed
./training/spam/00453.7ff1db57e1e39cb658661ef45b83a715
./training/spam/00454.ca6a81a702d62c23bc184a37a1cdb926
./training/spam/00455.8ccdcb205b6f8c3958bb3b2d39edca46
./training/spam/00456.c680a0c7d8d8d91bf3fb9f77ce6541b0
./training/spam/00457.f4325a4aa30dce61bf6c442b887733dd
./training/spam/00458.49467dba97d6ac2825c533df9a7a5017
./training/spam/00459.2c2341d0342bfedc94464025126bedc6
./training/spam/00460.407cd7d4ce577b1474eba6dd35a15081
./training/spam/00461.57e0fe5d31215393d7cf4e377e330082
./training/spam/00462.d0ca75a85184ca9843d1dffdc8c98855
./training/spam/00463.0bc4e08af0529dd773d9f10f922547db
./training/spam/00464.d2f719c667d192af860572b1c858cc11
./training/spam/00465.81b738fc646c03b1db38a456cd087ad7
./training

./training/spam/00599.d6d6a2edd58fa7dd6b18787e9867984b
./training/spam/00600.d113e8e0ac2d8d42f500f4faef61061b
./training/spam/00602.8b839b13833d7ca3abd6f6179ccc0286
./training/spam/00603.c76054a8ff1b75fb722602e567d2e7e1
./training/spam/00604.3dcf5f835dacff5d4a32a24eba31cbd2
./training/spam/00605.8a2e83e442d0052a2b2e9cff1ef0793c
./training/spam/00606.f078f6f48ae2e41f553882449f2aa613
./training/spam/00607.b5726a2ac74956ab9af0901379cdca5c
./training/spam/00608.bf7d77ef4f28552278f78a5aef71f0cf
./training/spam/00609.4dfe7912017772587dc62fecc3cf6553
./training/spam/00610.d78eda68ed03fa6890077bd4b805e16d
./training/spam/00611.58f9a1db64c83da2bfd3b1acf8d38338
./training/spam/00612.cd362b97ee34d41e72a66ed5199dd62e
./training/spam/00613.047cf0bcc74950bd9e1eaf8d336c385c
./training/spam/00614.974cdf353242f9286fbcc34673d9f28a
./training/spam/00615.e47bff6118d4ff6d98581fa6f40ab871
./training/spam/00616.1e86f2d3478f10a3a413844bba74f450
./training/spam/00617.f2097d6448725c371fd5f4154184ad3c
./training

...

./training/ham/0870.e7254d91187938f2f0b2fa1ff117f822
./training/ham/0871.79be1926ade2b8fc591f9f51abf66224
./training/ham/0872.b39e14fc64a28f30b5c413d51ed9dea1
./training/ham/0873.bd6a6b2911a0dedccee495aaf0fb248d
./training/ham/0874.f17f5355a2abf8cb83fb09069c9460bc
./training/ham/0875.990a5210c2586a027e13b10bf9d3e4ae
./training/ham/0876.22140b6b06918d04fc07c3d992a6b846
./training/ham/0877.62f5636ba5885d1b92423169c83a35b9
./training/ham/0878.24186e5267a8ec179a2e07f2da013932
./training/ham/0879.267af96ab014056d029ea42fd1ecf2bd
./training/ham/0880.6aa461d079bb0762af5c94d9af8487ec
./training/ham/0881.316c03b1dbd637537a4035f8470c6c12
./training/ham/0882.1fc35ed593366d26e06112250d18678a
./training/ham/0883.1c07a9bc574c386fbd893edbb24ea4e6
./training/ham/0884.f4a7181c5337229d1e70c587cbae9567
./training/ham/0885.edd07a1946446122321ba10a01eda39a
./training/ham/0886.a901020854d0c42772ba10e45212ee82
./training/ham/0887.95cafbddb7fce33aada0e9d9bb329aa5
./training/ham/0888.a8cf27198dd5b8612b47fe3f8f

./training/ham/1025.77dcf4ff92802ab3af94b9516d25b02a
./training/ham/1026.b2ee7b3cb90365641e465cfede58a672
./training/ham/1027.35033a975b9979f1a2eb34db590b32ac
./training/ham/1028.a139c4ec44f9dd8286a8333d934ced4e
./training/ham/1029.cfdcc9c1ab54e77b18da7164b324178a
./training/ham/1030.9a38ec315e2e8926085a560e36f977b8
./training/ham/1031.cfbae64b0894abd4ef88ddbf253fa704
./training/ham/1032.725446990feeb941bce8a383943cd2a2
./training/ham/1033.4155e06c0fb104b96ec6c5d656264fa7
./training/ham/1034.30d39e880274b0ebb59462ddfff16880
./training/ham/1035.a3cdb2fe04945379483b12640bdb19d4
./training/ham/1036.8c6e8c189738f671c5fd1cb2b1791317
./training/ham/1037.1285e23706448e51cb9be399cd0546ed
./training/ham/1038.4a0af891d4474c608eb6a3ffedeb8ada
./training/ham/1039.28e0f6b0f1e09bbcebc01183bd3557c4
./training/ham/1040.0975203bad39968cfceadfcd68a9242b
./training/ham/1041.c96194d26968c693aca4f4ef5f4a6a61
./training/ham/1042.5089423fc172a53a687d43ad6b432caf
./training/ham/1043.701d8a6b948c0e1406a85fc621

./training/ham/1180.9d441ee49eef8ba99c54ff8ec4ea9096
./training/ham/1181.e94c4c98963c99b42be08be341418d57
./training/ham/1182.3156d66c268869ee74f080a7c7523be8
./training/ham/1183.54b599ea32d753071ca44e55db7fe85d
./training/ham/1184.152999961bd0606fb0dcb6d6f95cc96d
./training/ham/1185.b83ac460b08616796da67d3bc57b76df
./training/ham/1186.fcd93eee19319006fece61d0d3a0bc13
./training/ham/1187.e5fee75dd86128e81186369567522f61
./training/ham/1188.1f6d5c1960b4d2d50e9745073559284e
./training/ham/1189.e69e458af7eefebc70d658dd2e6d23df
./training/ham/1190.99eaa82bada000ca0ada9512a3f590e2
./training/ham/1191.f8b145ebabbd42450c292e448cf44f15
./training/ham/1192.1d7f09a0119d74789f1918f9c02beb45
./training/ham/1193.8805a7218e81db9893ee2b704d9ebeb1
./training/ham/1194.ee628dd00ae3eea31232d0e78a39c9b2
./training/ham/1195.4cad51ea61eee58c9b4496ebba828692
./training/ham/1196.38c6bb13ea04559ae8d2cc19219d1ac5
./training/ham/1197.d2b11de4c8eb13e1fcc1d444665d0d1c
./training/ham/1198.b0a1f43fd2c1b88883e597b644

./training/ham/1335.57112a05c5a5ba633707ed3fdecfd88c
./training/ham/1336.7ea005648426a4f5815e0bb42230a595
./training/ham/1337.eb38bf81b8a24991252e451208c24ecd
./training/ham/1338.6656824f5801ca79629b3374e834ad04
./training/ham/1339.57de97732c190bf4575276eb054e0e79
./training/ham/1340.763507dad3b921f972640d4d76172f7e
./training/ham/1341.91bc30d50566e71807217c8977f7a793
./training/ham/1342.6770473802a7dc7ab14a212f95573c15
./training/ham/1343.c5ead41a4483bd3732e42d52c9163d24
./training/ham/1344.c12652263fc21b75dd4298a3d1bfcf53
./training/ham/1345.df3d6d8a5d9e6bba6484fbef2dae9102
./training/ham/1346.e172cd7f334b27ed0aeb288ca6436f7d
./training/ham/1347.b74ed7bb44086b2b4721bce965ac8273
./training/ham/1348.79cf2ab14db2301670710f9d1feaddda
./training/ham/1349.d05ca7b269c99915562d3e448f7b8afa
./training/ham/1350.e386afc23b51f446e2f6779c1ade0a68
./training/ham/1351.c176f7e5d80492c34cad40bc9e939012
./training/ham/1352.dde8603b55106a1e7fe9508e972b24d2
./training/ham/1353.bd9fa1be6b135e4aea436dc795

./training/ham/1490.e80cf700ef4de2385ada6ae149e7498c
./training/ham/1491.2e82a3803e51f420c6398d963052469f
./training/ham/1492.fcfeff3d31bfad092220ca731a40c05e
./training/ham/1493.0e19ac93ca8fe935323151417bc2e959
./training/ham/1494.adcdefb394b06836cbb548c4a93ee76a
./training/ham/1495.965bf18408e31d93d3fd256d2c6e8b64
./training/ham/1496.c48399a0c3abb60acd00322bdbb97565
./training/ham/1497.82b5d123cc0c3630a6ecd3d0eefe7c08
./training/ham/1498.578d31b0aab137aa9d79b455a6def5cd
./training/ham/1499.37d7a1b89dcf94a9b123ad584c2fa149
./training/ham/1500.2e6e497d8947d6125050b838efe4cf1f
./training/ham/1501.5c83f8ccf80ccff0b82b04a8e09d69ec
./training/ham/1502.1376704c3bd9c110fe3a5d0768745d91
./training/ham/1503.f1fc48f16902a097113fa60aaeb35245
./training/ham/1504.f77b2dc9ad8c875d8edc67b180e2f878
./training/ham/1505.7cc0d1e500937105c1503d63bd0b5161
./training/ham/1506.df1d0b609d034c834e290e7e3732e392
./training/ham/1507.381db954fc6fe8df9646172b53db92ad
./training/ham/1508.334b2eb6c70ba66605ebefc979

In [20]:
import time

# dumping vars to files (processing takes a long time, so it's better to do it once and save the vars into a file for later use)
f = open('./vars/corpus.pckl', 'rb')
corpus = pickle.load(f)
f.close()

f = open('./vars/vocab.pckl', 'rb')
vocab = pickle.load(f)
f.close()

f = open('./vars/labels.pckl', 'rb')
labels = pickle.load(f)
f.close()

#vectorization tf-idf style
vectorizer = TfidfVectorizer(vocabulary=vocab)
tfidf = vectorizer.fit_transform(corpus)
#print(tfidf.toarray())
print(len(vectorizer.get_feature_names()))
#print(vectorizer.idf_)


#SVM model
svr_rbf = SVR(kernel='rbf',C=10, gamma = 0.1)
start = time.time()
svr_rbf.fit(tfidf.toarray(), labels) 
end = time.time()
print('done in %.2f s' % (end - start))

12687
done in 65.45 s


In [21]:
# Cross-validation
ham_cv_corpus = []
spam_cv_corpus = []
ham_cv_path = './cross_validation/ham/'
spam_cv_path = './cross_validation/spam/'


# Testings
ham_test_corpus = []
spam_test_corpus = []
ham_test_path = './testing/ham/'
spam_test_path = './testing/spam/'


# read ham cross-validation data
i = 0
for filename in os.listdir(ham_cv_path):
    #i = i+1
    #if(i>302):
    #    break
    em = read_unlabeled_email((ham_cv_path+filename))
    if(em is None):
        continue
    ham_cv_corpus.append(em)

    
# read spam cross-validation data
i = 0
for filename in os.listdir(spam_cv_path):
    #i = i+1
    #if(i>302):
    #    break
    em = read_unlabeled_email((spam_cv_path+filename))
    if(em is None):
        continue
    spam_cv_corpus.append(em)

    
spam_cv_tfidf = vectorizer.fit_transform(spam_cv_corpus)
ham_cv_tfidf = vectorizer.fit_transform(ham_cv_corpus)

pred = svr_rbf.predict(spam_cv_tfidf.toarray())
print('performance on spam cross-validation set {0:.0%}'.format(sum(num >= 0.5 for num in pred)/len(pred)))

pred = svr_rbf.predict(ham_cv_tfidf.toarray())
print('performance on ham cross-validation set {0:.0%}'.format(sum(num < 0.5 for num in pred)/len(pred)))





performance on spam cross-validation set 97%
performance on ham cross-validation set 85%


In [22]:
# read ham test data
i = 0
for filename in os.listdir(ham_test_path):
    #i = i+1
    #if(i>302):
    #    break
    em = read_unlabeled_email((ham_test_path+filename))
    if(em is None):
        continue
    ham_test_corpus.append(em)

    
# read spam test data
i = 0
for filename in os.listdir(spam_test_path):
    #i = i+1
    #if(i>302):
    #    break
    em = read_unlabeled_email((spam_test_path+filename))
    if(em is None):
        continue
    spam_test_corpus.append(em)

    
spam_test_tfidf = vectorizer.fit_transform(spam_test_corpus)
ham_test_tfidf = vectorizer.fit_transform(ham_test_corpus)

pred = svr_rbf.predict(spam_test_tfidf.toarray())
print('performance on spam test set {0:.0%}'.format(sum(num >= 0.5 for num in pred)/len(pred)))

pred = svr_rbf.predict(ham_test_tfidf.toarray())
print('performance on ham test set {0:.0%}'.format(sum(num < 0.5 for num in pred)/len(pred)))



performance on spam test set 97%
performance on ham test set 93%
